In [ ]:
from datascience import *
from pandas import read_stata
import numpy as np

import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)


In [ ]:
def minimize_multi(f, start=None, **vargs):
  def expanded_f(*args):
    return f(args)
  return minimize(expanded_f, start=start, method="L-BFGS-B", **vargs)

In [ ]:
data = Table.read_table('Young Lives R1....csv')
data.show(5)

In [ ]:
# Remove nans from pertinent columns
dataframe = data.to_df()
altered = dataframe.fillna(-99)
data = Table.from_df(altered)
data.show(5)

## Multiple Regression

<font color="Blue"> Another way to predict outcomes is with multivariate regression.  Prepare two multivariate regressions to compare: one regression will have all the variables you think are important and the other will have variables you think are not important.  Compare the R^2 of these regressions.  Were you right? Explain.

We will look at the steps to building multivariate regression examining how age and sex predict height.

In [ ]:
# Make sure all your data is in integer format and keep only observations with height, age & sex data
data["chheight_r1"]=data.apply(int, 'chheight_r1')
data["agechild_r1"]=data.apply(int, 'agechild_r1')
data["agemum_r1"]=data.apply(int, 'agemum_r1')
# Make a column that will be associated with the intercept
data["For Intercept"]=1

# Select all the variables you want to analyze including intercept and outcome.
# (Until we have the correct minimizing function, just include one variable and the outcome.)
Regression1=data.select("...","...","...","For Intercept")



In [ ]:
#remove missing
Regression1=Regression1.where(Regression1['...']>0)
Regression1=Regression1.where(Regression1['...']>0)
Regression1=Regression1.where(Regression1['...']>0)
Regression1

In [ ]:
# Divide the table in two: predictor variables (sex & age) & outcome variable (height)
X_features_table =Regression1.drop("chheight_r1")
X_true_values = Regression1.column("chheight_r1") # note: this is an array
X_features_table

In [ ]:
# Choose some intial values for maximizing the coefficients 
# An easy choice is all 0's.  
# There will be the same number of coefficients as there are predictors - a slope for each predictor.
X_initial_coefficient_guess = np.zeros(X_features_table.num_columns)
X_initial_coefficient_guess

In [ ]:
def predict_all(features_table, coefficients):
    """
    Given a table of features called features_table and some coefficients,
    produces linear predictions for each row of features_table.
    
    features_table should be a table with one column for each feature
    being used to predict.  Each row represents a house in the task
    we're doing in this lab.
    
    coefficients should be an array with one element for each column in
    features_table, like the coefficients computed by the function
    least_squares_coefficients.
    
    Returns an array of predictions, one for each row of features_table.
    
    If there were 3 rows in features_table, we would return a 3-element
    array instead, containing the predicted prices for each row.
    """
   # assert features_table.num_columns == len(coefficients), /
    """
    The first argument to predict_all should be a table with one
    column for each feature.  That means it should have the same
    number of columns as the coefficients array (the second
    argument) has elements.
    """
    def predict(features):
        # Given an array of features, produce one prediction.
        return sum(features * coefficients)
    predictions = Table().with_column('features', features_table.rows).apply(predict, 'features')
    return predictions

In [ ]:
X_predictions=predict_all(X_features_table, X_initial_coefficient_guess)
X_predictions

In [ ]:
# change the initial coefficient guess up above to make a different prediction!
X_new_coefficient_guess = [1]
X_predictions=predict_all(X_features_table, X_new_coefficient_guess)
X_predictions

In [ ]:
def compute_errors(features_table, coefficients, true_values):
    """
    Computes the prediction errors for a linear model with the given
    coefficients when predicting the true values for the given
    examples.
    
    features_table should be a table with one column for each feature
    being used to predict.  
    
    coefficients should be an array of numbers, one for each feature.
    
    true_values should be an array of numbers, one for each row in
    features_table.  It records the true prices of each house.
    """
    return predict_all(features_table, coefficients) - true_values



In [ ]:
X_errors=compute_errors(X_features_table, X_initial_coefficient_guess, X_true_values)
X_errors

In [ ]:
def rmse(errors):
    """
    Computes the root mean squared error when a regression model makes
    the given errors.  So errors should be an array of numbers, one for
    each row in some data table for which we're computing predictions.  
    Each number is the prediction error of some regression model.
    """
    return np.mean(errors**2)**0.5



In [ ]:
r2=rmse(X_errors)
r2

In [ ]:
def make_least_squares_objective_function(features_table, true_values):
    """
    Makes an objective function for main data in the features_table
    table, where the true values we're trying to predict are true_values.
    
    features_table should be a table with one column for each feature
    being used to predict.  
    
    true_values should be an array of numbers, one for each row in
    features_table.  
    
    The returned value is a function.  That function takes an array of
    coefficients and returns a number.  Larger values of that number
    mean that those coefficients produce worse prediction errors.
    """
    def objective_function(coefficients):
        errors = compute_errors(features_table, np.array(coefficients), true_values)
        return rmse(errors)
    return objective_function



In [ ]:
X_objective_function = make_least_squares_objective_function(X_features_table, X_true_values)
X_objective_function

In [ ]:
def least_squares_coefficients(main_data, predicted_column_name):
    """
    Performs multiple linear regression predicting predicted_column_name
    using the other columns of main_data as features.
    
    main_data should be a table with one column for each feature
    being used to predict, plus one column for the value we're trying
    to predict.  That column's name should equal predicted_column_name.
    Each row represents a house in the task we're doing in this lab.
    
    predicted_column_name should be a string, the name of the column in
    main_data that we're trying to predict.
    
    Returns an array of coefficients, one for each feature (that is, one
    for each column in main_data other than predicted_column_name).
    
    For example, if main_data has 3 columns, mom's education, age, and height,
    and predicted_column_name is "Price", then we will use mom's education and
    age to predict height.  This function will return an array of 2
    numbers, a regression coefficient for mom's education and a
    regression coefficient for age.
    """
    features_table = main_data.drop(predicted_column_name)
    true_values = main_data.column(predicted_column_name)
    objective_function_OLS = make_least_squares_objective_function(features_table, true_values)
    
    # Now we find the coefficients that produce the smallest
    # error.
    initial_coefficient_guess = np.zeros(features_table.num_columns)
    best_coefficients = minimize_multi(objective_function_OLS, start=initial_coefficient_guess)
    if features_table.num_columns == 1:
        return np.array([best_coefficients])
    else:
        return best_coefficients

In [ ]:
X_best_coefficients = minimize_multi(X_objective_function, start=X_initial_coefficient_guess)
X_best_coefficients

In [ ]:
My_Coefficients = least_squares_coefficients(Regression1, '...')
My_Coefficients

Let's see how well our model fits the data.  We will calculate the R^2.

In [ ]:
mean_height=np.average(Regression1["..."])
mean_height

In [ ]:
My_predictions=predict_all(X_features_table, My_Coefficients)
Regression1["Predicted"]=My_predictions
Regression1["Diff_Predict_SQ"]=(Regression1["..."]-Regression1["Predicted"])**2
Regression1["Diff_True_SQ"]=(Regression1["..."]-mean_height)**2
Regression1

In [ ]:
r_squared=1-np.sum(Regression1["Diff_Predict_SQ"])/np.sum(Regression1["Diff_True_SQ"])
r_squared

Let's be more systematic about building the model - we will add variables to try to get the largest R^2.  Adding more covariates will improve the fit of the model, but we are also making the model more complex.  Let's add covariates in order of importance, keeping those that increase the adjusted R^2.  The formula for the adjusted R^2 is 1-(1-R^2)(N-1)/(N-p-1), where N= sample size and p= number of predictors.

This model is of correlation, not causation.  But since this is a first indication of a potential causal relationship, we may as well try a policy and then later see if it works out. <font color="Blue"> If you were attempting to change the outcome based on changing the feature, which feature would you try to change?  Don't just consider the one with the highest correlation, but also take into account costs and difficulty of changing the feature. (Not a data exercise)